In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv(r"C:\Users\Maria Paula\OneDrive\Documents\GitHub\airemty\base_final_imputada_por_zona.csv")
df = pd.DataFrame(data)

In [ ]:
df.dtypes

In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['year'] = df['date'].dt.year #extraigo el año

In [ ]:
df_noche = df[(df["date"].dt.hour >= 20) | (df["date"].dt.hour <= 6)]
df_agrupado = df_noche.groupby(["Zona", "year"])


In [ ]:
'''for (zona, año), grupo in df_agrupado:
    print(f"Zona: {zona}, Año: {año}, Registros: {len(grupo)}")
    display(grupo.head(1))'''

In [ ]:
df_estadisticas = (
    df_agrupado['SO2']
    .agg(['mean', 'median', 'std', 'min', 'max'])
    .reset_index()
)

# Si quieres valores absolutos de la media (opcional)
df_estadisticas['mean'] = df_estadisticas['mean'].abs()
df_estadisticas['median'] = df_estadisticas['median'].abs()
df_estadisticas['std'] = df_estadisticas['std'].abs()
df_estadisticas['min'] = df_estadisticas['min'].abs()

In [ ]:
import matplotlib.pyplot as plt

df_estadisticas[['mean', 'median', 'std', 'min', 'max']].hist(
    bins=20,
    figsize=(10, 8),
    grid=False
)

plt.suptitle("Distribución de medidas estadísticas por Zona y Año de SO2", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
df_estadisticas

In [ ]:
from tslearn.metrics import cdist_dtw
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

FEATURES = ['mean', 'median', 'std', 'min', 'max']

dfs_por_anio = {}
for y in sorted(df_estadisticas['year'].unique()):
    sub = (
        df_estadisticas.loc[df_estadisticas['year'] == y, ['Zona', 'year'] + FEATURES]
        .drop_duplicates(subset=['Zona'])        # por si hay repetidas
        .set_index('Zona')                       # etiquetas del dendrograma
    )
    # Quédate con columnas numéricas (por si se coló algo)
    sub_num = sub.select_dtypes(include='number')
    dfs_por_anio[y] = sub_num
    
df_2022 = dfs_por_anio[2022].select_dtypes(include='number')
df_2021 = dfs_por_anio[2021].select_dtypes(include='number')
df_2023 = dfs_por_anio[2023].select_dtypes(include='number')
df_2024 = dfs_por_anio[2024].select_dtypes(include='number')
df_2025 = dfs_por_anio[2025].select_dtypes(include='number')


In [ ]:
df_2023

In [ ]:
# Matriz de distancias DTW
dist_matrix = cdist_dtw(df_2021)

# Clustering jerárquico
Z = linkage(dist_matrix, method='ward', metric='euclidean')

dendrogram(Z, labels=df_2021.index.tolist(), leaf_rotation=90, leaf_font_size=10)
plt.title("Dendrograma jerárquico — Año 2021 (métrica: DTW)")
plt.ylabel("Distancia")
plt.show()

In [ ]:
# Matriz de distancias DTW
dist_matrix_2 = cdist_dtw(df_2022)

# Clustering jerárquico
Z_2 = linkage(dist_matrix_2, method='ward', metric='euclidean')

dendrogram(Z_2, labels=df_2022.index.tolist(), leaf_rotation=90, leaf_font_size=10)
plt.title("Dendrograma jerárquico — Año 2022 (métrica: DTW)")
plt.ylabel("Distancia")
plt.show()

In [ ]:
# Matriz de distancias DTW
dist_matrix_3 = cdist_dtw(df_2023)

# Clustering jerárquico
Z_3 = linkage(dist_matrix_3, method='ward', metric='euclidean')

dendrogram(Z_3, labels=df_2023.index.tolist(), leaf_rotation=90, leaf_font_size=10)
plt.title("Dendrograma jerárquico — Año 2023 (métrica: DTW)")
plt.ylabel("Distancia")
plt.show()

In [ ]:
# Matriz de distancias DTW
dist_matrix_4 = cdist_dtw(df_2024)

# Clustering jerárquico
Z_4 = linkage(dist_matrix_4, method='ward', metric='euclidean')

dendrogram(Z_4, labels=df_2024.index.tolist(), leaf_rotation=90, leaf_font_size=10)
plt.title("Dendrograma jerárquico — Año 2024 (métrica: DTW)")
plt.ylabel("Distancia")
plt.show()

In [ ]:
# Matriz de distancias DTW
dist_matrix_5 = cdist_dtw(df_2025)

# Clustering jerárquico
Z_5 = linkage(dist_matrix_5, method='ward', metric='euclidean')

dendrogram(Z_5, labels=df_2025.index.tolist(), leaf_rotation=90, leaf_font_size=10)
plt.title("Dendrograma jerárquico — Año 2025 (métrica: DTW)")
plt.ylabel("Distancia")
plt.show()

# K óptimo

In [ ]:
import numpy as np
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

years = [2021, 2022, 2023, 2024, 2025]
dist_mats = {
    2021: dist_matrix,
    2022: dist_matrix_2,
    2023: dist_matrix_3,
    2024: dist_matrix_4,
    2025: dist_matrix_5,
}
links = {2021: Z, 2022: Z_2, 2023: Z_3, 2024: Z_4, 2025: Z_5}

# Rango de k a evaluar
k_min, k_max = 2, 10
ks = range(k_min, k_max + 1)

sil_means, sil_stds = [], []

labels_by_year_k = {y: {} for y in years}

for k in ks:
    sil_year = []
    for y in years:
        Z = links[y]
        D = dist_mats[y]
        # Cortar el dendrograma
        labels = fcluster(Z, k, criterion='maxclust')
        labels_by_year_k[y][k] = labels
        # Silhouette con distancias precomputadas
        s = silhouette_score(D, labels, metric='precomputed')
        sil_year.append(s)
    sil_means.append(np.mean(sil_year))
    sil_stds.append(np.std(sil_year, ddof=1))

# Selección de k*: mayor media y, si empata, menor desviación
sil_means = np.array(sil_means)
sil_stds = np.array(sil_stds)

best_idx = np.lexsort((sil_stds * 1.0, -sil_means))  # ordena por (-mean, std)
k_star = list(ks)[best_idx[0]]

print(f"Mejor k común (media Silhouette alta y estable): {k_star}")
for k, m, s in zip(ks, sil_means, sil_stds):
    print(f"k={k:2d}  Silhouette mean={m:.3f}  std={s:.3f}")

# (Opcional) Grafiquito para ver el trade-off
plt.figure(figsize=(6,4))
plt.errorbar(list(ks), sil_means, yerr=sil_stds, fmt='-o')
plt.axvline(k_star, linestyle='--')
plt.title("Silhouette promedio ± sd por k (2021–2025)")
plt.xlabel("k")
plt.ylabel("Silhouette (metric='precomputed' con DTW)")
plt.tight_layout()
plt.show()

# Clusters por año

In [ ]:
from scipy.cluster.hierarchy import fcluster

labels = fcluster(Z, 2, criterion='maxclust')  # fuerza 2 clústeres
labels_2 = fcluster(Z_2, 2, criterion='maxclust')  
labels_3 = fcluster(Z_3, 2, criterion='maxclust')  
labels_4 = fcluster(Z_4, 2, criterion='maxclust')  
labels_5 = fcluster(Z_5, 2, criterion='maxclust')

import pandas as pd
Clust_anio = pd.DataFrame({
    'objeto': df_2025.index,
    'cluster_2021': labels,
    'cluster_2022': labels_2,
    'cluster_2023': labels_3,
    'cluster_2024': labels_4,
    'cluster_2025': labels_5,
}).set_index('objeto')
Clust_anio

In [ ]:
import matplotlib.pyplot as plt

# --- Asumo tu tabla tal cual la mostraste:
# Clust_anio: index = objeto, columnas = cluster_2021 ... cluster_2025 con valores 1/2
Clust_graf = Clust_anio.copy()

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.heatmap(Clust_graf[["cluster_2021",	"cluster_2022",	"cluster_2023",	"cluster_2024",	"cluster_2025"]], annot=True, fmt=".0f", cbar=False, linewidths=.5)
plt.title("Mapa de clúster por objeto y año")
plt.xlabel("Año")
plt.ylabel("Objeto")
plt.tight_layout()
plt.show()


# Gráficas en mapa de las estaciones y la refineria

In [ ]:
import folium
from folium import Circle, Marker

In [ ]:
data_ref = pd.read_csv(r'C:\Users\Maria Paula\OneDrive\Documents\GitHub\airemty\distancias_a_refineria.csv')
df_ref = pd.DataFrame(data_ref)
df_ref.head()

In [ ]:
import folium
from folium import Circle, Marker

# Coordenadas de la Refinería de Cadereyta
ref_lat, ref_lon = 25.59056, -100.00139

# Crear mapa centrado en la refinería
mapa = folium.Map(location=[ref_lat, ref_lon], zoom_start=9, tiles="OpenStreetMap")

# Agregar marcador de la refinería
Marker(
    location=[ref_lat, ref_lon],
    popup="Refinería de Cadereyta",
    icon=folium.Icon(color="red", icon="industry", prefix="fa")
).add_to(mapa)

# Agregar círculo de referencia (50 km)
Circle(
    location=[ref_lat, ref_lon],
    radius=50000,  # 50 km
    color="blue",
    fill=True,
    fill_opacity=0.05,
    popup="Radio 50 km de la Refinería"
).add_to(mapa)

# Recorrer filas del DataFrame df_ref
for idx, row in df_ref.iterrows():
    elev = row['elev_m']
    delta = row['delta_elev_m']
    
    # Definir color según delta de elevación
    if delta < 0:
        color = "blue"
    elif delta < 150:
        color = "green"
    elif delta < 250:
        color = "orange"
    else:
        color = "red"
    
    # Agregar marcador de estación
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=folium.Popup(
            f"<b>{row['station']}</b><br>"
            f"Elevación: {elev} m<br>"
            f"ΔElev: {delta:+} m<br>"
            f"Distancia: {row['distance_km']:.1f} km",
            max_width=250
        )
    ).add_to(mapa)

# Mostrar el mapa
mapa


# Animación Clusters por año

In [ ]:
import folium
from folium import Circle, Marker
from folium.plugins import TimestampedGeoJson
import pandas as pd
from datetime import datetime

# ---------------------------
# INPUTS
# ---------------------------
# Years present in your cluster columns
years = [2021, 2022, 2023, 2024, 2025]

# Map cluster → color (change if you add more clusters)
cluster_color = {1: "#b41f1f",  # 
                 2: "#fffb0e"}  # 

# ---------------------------
# BASE MAP (your original bits)
# ---------------------------
ref_lat, ref_lon = 25.59056, -100.00139
m = folium.Map(location=[ref_lat, ref_lon], zoom_start=9, tiles="OpenStreetMap")

Marker(
    location=[ref_lat, ref_lon],
    popup="Refinería de Cadereyta",
    icon=folium.Icon(color="red", icon="industry", prefix="fa")
).add_to(m)

Circle(
    location=[ref_lat, ref_lon],
    radius=50000,  # 50 km
    color="blue",
    fill=True,
    fill_opacity=0.05,
    popup="Radio 50 km de la Refinería"
).add_to(m)

# ---------------------------
# BUILD A TIME-ENABLED GEOJSON
# ---------------------------
# Helper to pick color from your delta elev logic (optional; used in popup info)
def elev_color(delta):
    if delta < 0:
        return "blue"
    elif delta < 150:
        return "green"
    elif delta < 250:
        return "orange"
    return "red"

features = []
# --- 1) Diccionario: station (df_ref) -> objeto (Clust_anio) ---
# AJÚSTALO a tus nombres reales:
station_to_obj = {
    "CE": "Centro",
    "NTE": "Norte",      "NTE2": "Norte2",
    "NE": "Noreste",   "NE2": "Noreste2",   "NE3": "Noreste3",
    "NO": "Noroeste",  "NO2": "Noroeste2",  "NO3": "Noroeste3",
    "SUR": "Sur",
    "SE": "Sureste",   "SE2": "Sureste2",   "SE3": "Sureste3",
    "SO": "Suroeste",  "SO2": "Suroeste2",  "SO3": "Suroeste3",
    
}

# --- 2) Prepara Clust_anio para unir ---
# Si Clust_anio tiene 'objeto' como índice, pásalo a columna
if "objeto" not in Clust_anio.columns:
    Clust_anio = Clust_anio.reset_index()  # ahora tiene columna 'objeto'
    if "objeto" not in Clust_anio.columns and "index" in Clust_anio.columns:
        Clust_anio = Clust_anio.rename(columns={"index": "objeto"})

cluster_cols = ["cluster_2021","cluster_2022","cluster_2023","cluster_2024","cluster_2025"]

df_ref["objeto"] = df_ref["station"].map(station_to_obj)

# (opcional) ver qué estaciones no se mapearon y arreglar el diccionario si sale algo
faltan = df_ref[df_ref["objeto"].isna()]["station"].unique().tolist()
if faltan: print("⚠️ Estaciones sin mapeo:", faltan)

# 2) Ahora sí: merge por 'objeto'
cluster_cols = ["cluster_2021","cluster_2022","cluster_2023","cluster_2024","cluster_2025"]
df_all = df_ref.merge(Clust_anio[["objeto"] + cluster_cols], on="objeto", how="left")

for y in years:
    # a fake timestamp per year (TimestampedGeoJson needs a time)
    ts = datetime(y, 1, 1).strftime("%Y-%m-%dT%H:%M:%S")

    # the cluster column name, e.g., cluster_2021
    ccol = f"cluster_{y}"
    for _, r in df_all.iterrows():
        cl = int(r[ccol])
        color = cluster_color.get(cl, "#999999")

        # Build one Feature per station per year
        feat = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [float(r["lon"]), float(r["lat"])]
            },
            "properties": {
                "time": ts,
                "icon": "circle",
                "iconstyle": {
                    "fillColor": color,
                    "fillOpacity": 0.9,
                    "stroke": True,
                    "color": color,
                    "weight": 1,
                    "radius": 7
                },
                "popup": (
                    f"<b>{r['station']}</b><br>"
                    f"Año: {y}<br>"
                    f"Cluster: {cl}<br>"
                    f"Elevación: {r['elev_m']} m<br>"
                    f"ΔElev: {r['delta_elev_m']:+} m "
                    f"({elev_color(r['delta_elev_m'])})<br>"
                    f"Distancia: {r['distance_km']:.1f} km"
                ),
                # style for non-point (not used here),
                # but TimestampedGeoJson expects style keys
                "style": {"color": color, "weight": 1}
            }
        }
        features.append(feat)

geojson = {
    "type": "FeatureCollection",
    "features": features
}

TimestampedGeoJson(
    data=geojson,
    transition_time=400,    # ms between frames
    period="P1Y",           # 1-year steps
    add_last_point=False,
    auto_play=False,
    loop=False,
    max_speed=1,            # slider max
    loop_button=True,
    date_options="YYYY",    # show just the year on the control
    time_slider_drag_update=True
).add_to(m)

# Optional legend
legend_html = """
<div style='position: fixed; bottom: 30px; left: 30px; z-index:9999; 
     background: white; padding: 10px 12px; border: 1px solid #ccc; 
     border-radius: 6px; font-size: 14px;'>
<b>Clusters</b><br>
<span style='display:inline-block;width:12px;height:12px;background:#b41f1f;margin-right:6px;'></span> 1<br>
<span style='display:inline-block;width:12px;height:12px;background:#fffb0e;margin-right:6px;'></span> 2
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

m  # display in Jupyter / save as HTML with m.save("clusters_anim.html")


# Medidas descriptivas por Cluster

In [ ]:
for y in [2021, 2022, 2023, 2024, 2025]:
    ccol = f"cluster_{y}"
    df_year = globals()[f"df_{y}"]  # accede a df_2021, df_2022, etc.
    df_year = df_year.merge(Clust_anio[["objeto", ccol]], left_on="Zona", right_on="objeto", how="left")
    globals()[f"df_{y}"] = df_year  # actualiza el dataframe en memoria


In [ ]:
# Diccionario para guardar los resultados de cada año
resumenes = {}

# Bucle para calcular automáticamente el resumen de cada año
for year in [2021, 2022, 2023, 2024, 2025]:
    df = globals().get(f"df_{year}")  # obtiene la variable df_2021, df_2022, etc.
    
    if df is not None:  # verifica que exista
        resumen = (
            df.groupby(f'cluster_{year}')
              .agg({
                  'mean': ['mean', 'std'],
                  'median': ['mean'],
                  'std': ['mean'],
                  'min': ['mean'],
                  'max': ['mean']
              })
        )
        # Aplana las columnas para que sean más legibles
        resumen.columns = ['_'.join(col) for col in resumen.columns]
        resumen.reset_index(inplace=True)
        
        # Guarda el resultado
        resumenes[year] = resumen

In [ ]:
res_2021 = resumenes[2021]
res_2022 = resumenes[2022]
res_2023 = resumenes[2023]
res_2024 = resumenes[2024]
res_2025 = resumenes[2025]

In [ ]:
resumen_todos = pd.concat(
    [res_2021.assign(year=2021),
     res_2022.assign(year=2022),
     res_2023.assign(year=2023),
     res_2024.assign(year=2024),
     res_2025.assign(year=2025)],
    ignore_index=True
)

# Mostrar el resultado medias de medidas estadisticas de concentracion de SO2
resumen_todos[[	'year','mean_mean',	'mean_std',	'median_mean',	'std_mean',	'min_mean',	'max_mean']]

# Promedio de distancia de estaciones por cluster a refineria

In [ ]:
import pandas as pd

# Lista de columnas de cluster por año
cluster_cols = ["cluster_2021", "cluster_2022", "cluster_2023", "cluster_2024", "cluster_2025"]

# Diccionario donde se guardará el resultado
dist_med_porclust = {}

# Calcular la distancia promedio por cluster para cada año
for col in cluster_cols:
    # Agrupamos por el cluster de ese año y calculamos la media de las distancias
    dist_med = df_all.groupby(col)['distance_km'].mean()
    
    # Guardamos el resultado
    dist_med_porclust[col] = dist_med

# Convertimos el resultado a un DataFrame para ver todo junto
dist_med_porclust_df = pd.DataFrame(dist_med_porclust)
dist_med_porclust_df


# Frecuencia de Outliers

In [2]:
import pandas as pd
data_sinimp = pd.read_csv(r'C:\Users\Maria Paula\OneDrive\Documents\GitHub\airemty\base_final_sin_imputar_por_zona.csv')
raw_data = pd.DataFrame(data_sinimp)

In [3]:
# 0) (opcional) quedarte sólo con 20:00–06:00
raw_data["date"] = pd.to_datetime(raw_data["date"], errors="coerce")
raw_data["year"] = raw_data["date"].dt.year
df_noche_raw = raw_data[(raw_data["date"].dt.hour >= 20) | (raw_data["date"].dt.hour <= 6)]
df_noche_raw = df_noche_raw[(df_noche_raw["date"].dt.year > 2020)]

In [ ]:
df_noche_raw.head(10)

In [ ]:
import pandas as pd
import numpy as np

# ----------------------------
# 1) Helpers
# ----------------------------
ALIASES = {
    "TOUT": "TEMP",
    "WSR":  "WS",
    "PRS":  "BP",
    "NOX":  "NOx",
    "Rainf": "RAINF",
    "RainF": "RAINF",
}

MFG_RANGES = {
    "PM10":  (0, 1000),
    "PM2.5": (0, 1000),
    "O3":    (0, 1000),
    "NO":    (0, 500),
    "NO2":   (0, 500),
    "NOx":   (0, 500),
    "SO2":   (0, 500),
    "CO":    (0, 50),
    "RH":    (0, 100),
    "WS":    (0, 180),
    "TEMP":  (-50, 50),
    "SR":    (0, 1.4),
    "BP":    (49.9, 824),
    "WD":    (0, 360),
    "RAINF": (0, None),  # solo validamos que no sea negativo
}

OP_RANGES = {
    2021: {"PM10":(0,800),"PM2.5":(0,325),"O3":(0,175),"NO":(0,350),"NO2":(0,100),
           "NOx":(0,400),"SO2":(0,150),"CO":(0,10),"RH":(0,100),"WS":(0,40),
           "TEMP":(-6.5,45),"SR":(0,1.0),"BP":(690,740),"WD":(0,360),"RAINF":(0,80)},
    2022: {"PM10":(0,999),"PM2.5":(0,450),"O3":(0,160),"NO":(0,400),"NO2":(0,175),
           "NOx":(0,420),"SO2":(0,150),"CO":(0,10),"RH":(0,100),"WS":(0,35),
           "TEMP":(-5,45),"SR":(0,1.25),"BP":(700,740),"WD":(0,360),"RAINF":(0,25)},
    2023: {"PM10":(0,900),"PM2.5":(0,800),"O3":(0,175),"NO":(0,500),"NO2":(0,175),
           "NOx":(0,500),"SO2":(0,150),"CO":(0,14),"RH":(0,100),"WS":(0,40),
           "TEMP":(0,45),"SR":(0,1.0),"BP":(690,740),"WD":(0,360),"RAINF":(0,70)},
    2024: {"PM10":(0,999),"PM2.5":(0,999),"O3":(0,180),"NO":(0,400),"NO2":(0,130),
           "NOx":(0,500),"SO2":(0,150),"CO":(0,18),"RH":(0,100),"WS":(0,38),
           "TEMP":(-4,45.5),"SR":(0,1.26),"BP":(687.5,740),"WD":(0,360),"RAINF":(0,50)},
    2025: {"PM10":(0,820),"PM2.5":(0,350),"O3":(0,185),"NO":(0,350),"NO2":(0,175),
           "NOx":(0,400),"SO2":(0,150),"CO":(0,10),"RH":(0,100),"WS":(0,40),
           "TEMP":(-4.5,45),"SR":(0,1.2),"BP":(688,740),"WD":(0,360),"RAINF":(0,25)},
}

def _norm_cols(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out.columns = [str(c).strip() for c in out.columns]
    out.rename(columns={k: v for k, v in ALIASES.items() if k in out.columns}, inplace=True)
    return out

def _to_num(df: pd.DataFrame, cols=None) -> pd.DataFrame:
    out = df.copy()
    if cols is None:
        # intenta todas salvo no-numéricas típicas
        cols = [c for c in out.columns if c not in ("Zona","date","station","Estacion")]
    out.replace({"NULL": np.nan, "NaN": np.nan, "nan": np.nan, "": np.nan}, inplace=True)
    for c in cols:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

def _apply_mfg_filter(df: pd.DataFrame) -> pd.DataFrame:
    mask = pd.Series(True, index=df.index)
    for var, (vmin, vmax) in MFG_RANGES.items():
        if var not in df.columns: 
            continue
        m = pd.Series(True, index=df.index)
        if vmin is not None:
            m &= (df[var].isna()) | (df[var] >= vmin)
        if vmax is not None:
            m &= (df[var].isna()) | (df[var] <= vmax)
        mask &= m
    return df[mask].copy()

def _year_bounds(var: str):
    y2min, y2max = {}, {}
    for y, d in OP_RANGES.items():
        if var in d:
            y2min[y] = d[var][0]
            y2max[y] = d[var][1]
    return y2min, y2max


In [ ]:
# ----------------------------
# 2) Pipeline
# ----------------------------
# Normaliza columnas & convierte a numérico
df1 = _norm_cols(df_noche_raw)
df1["date"] = pd.to_datetime(df1["date"], errors="coerce")
df1["year"] = df1["date"].dt.year

num_cols = [c for c in df1.columns if c not in ("Zona","station","Estacion","date")]
df1 = _to_num(df1, num_cols)

# Filtra por rangos del fabricante
df_clean = _apply_mfg_filter(df1)

# ----------------------------
# 3) Outliers por AÑO contra OP_RANGES (solo SO2 y TEMP)
# ----------------------------
for VAR in ["SO2", "TEMP"]:
    if VAR not in df_clean.columns:
        # si falta alguna, crea col vacía para que el groupby no truene
        df_clean[VAR] = np.nan

    y2min, y2max = _year_bounds(VAR)
    df_clean[f"{VAR}_min"] = df_clean["year"].map(y2min)
    df_clean[f"{VAR}_max"] = df_clean["year"].map(y2max)

    # outlier si está fuera del [min, max] del año correspondiente
    df_clean[f"out_{VAR}"] = (
        (~df_clean[VAR].isna()) &
        (
            (df_clean[VAR] < df_clean[f"{VAR}_min"]) |
            (df_clean[VAR] > df_clean[f"{VAR}_max"])
        )
    )

# ----------------------------
# 4) Tabla de frecuencias por Zona-año
# ----------------------------
# Nombre de la columna de estación: usa 'Zona' si existe, si no 'Estacion' o 'station'
station_col = "Zona" if "Zona" in df_clean.columns else ("Estacion" if "Estacion" in df_clean.columns else "station")

freq = (
    df_clean
    .groupby([station_col, "year"], dropna=False)
    .agg(
        total_obs = ("SO2", "size"),
        so2_out   = ("out_SO2", "sum"),
        temp_out  = ("out_TEMP", "sum")
    )
    .reset_index()
)

# Porcentajes
freq["so2_out_%"]  = (freq["so2_out"]  / freq["total_obs"] * 100).round(2)
freq["temp_out_%"] = (freq["temp_out"] / freq["total_obs"] * 100).round(2)

# (Opcional) outlier en cualquiera de los dos
freq["any_out"]    = freq["so2_out"] + freq["temp_out"]
freq["any_out_%"]  = (freq["any_out"] / freq["total_obs"] * 100).round(2)

# Ordena para lectura
freq = freq.sort_values([station_col, "year"]).reset_index(drop=True)

# Resultado principal:
freq_nonzero = freq[(freq["so2_out"] > 0) | (freq["temp_out"] > 0)]
freq_nonzero.head(50)

In [5]:
df_noche_raw

,Zona,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,year
8777,CENTRO,2021-01-01 00:00:00,4.95,NaN,NaN,NaN,22.0,68.0,64.10,NaN,NaN,64.0,3.2,0.000,9.05,3.7,336.0,2021.0
8778,CENTRO,2021-01-01 01:00:00,4.73,NaN,NaN,NaN,27.0,68.0,46.00,709.7,0.00,67.0,3.3,0.167,8.26,7.9,78.0,2021.0
8779,CENTRO,2021-01-01 02:00:00,4.72,NaN,NaN,NaN,23.0,55.0,47.64,710.1,0.00,70.0,3.3,0.165,7.15,4.8,47.0,2021.0
8780,CENTRO,2021-01-01 03:00:00,4.82,15.8,15.8,NaN,25.0,57.0,52.07,710.4,0.00,73.0,3.6,0.164,6.20,5.2,349.0,2021.0
8781,CENTRO,2021-01-01 04:00:00,5.02,NaN,NaN,NaN,25.0,65.0,61.62,710.6,0.00,74.0,3.5,0.164,5.64,3.2,358.0,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697127,SUROESTE2,2025-06-30 06:00:00,0.18,3.2,6.0,9.3,17.0,22.0,NaN,713.7,0.00,74.0,3.5,0.000,22.07,6.8,206.0,2025.0
697141,SUROESTE2,2025-06-30 20:00:00,0.48,6.4,12.5,18.9,11.0,22.0,9.00,713.6,0.00,86.0,3.3,0.006,21.87,11.9,191.0,2025.0
697142,SUROESTE2,2025-06-30 21:00:00,0.36,4.6,9.3,14.0,11.0,19.0,NaN,713.7,0.01,86.0,3.1,0.000,21.68,13.0,190.0,2025.0
697143,SUROESTE2,2025-06-30 22:00:00,0.32,4.4,7.8,12.2,14.0,24.0,NaN,713.7,0.00,84.0,3.2,0.000,21.81,11.6,192.0,2025.0


In [ ]:
''''import pandas as pd
import numpy as np

# --- base de cálculo (ajusta si no usas df_clean) ---
base = df_noche_raw.copy()

# Detecta nombre de columna de estación
station_col = "Zona" if "Zona" in base.columns else ("Estacion" if "Estacion" in base.columns else "station")

# Asegura fecha y columnas numéricas
base["date"] = pd.to_datetime(base["date"], errors="coerce")
for c in ["SO2", "TOUT"]:
    if c in base.columns:
        base[c] = pd.to_numeric(base[c], errors="coerce")

# Deriva año/mes
base["year"]  = base["date"].dt.year
base["month"] = base["date"].dt.month
base["month_name"] = base["date"].dt.month_name(locale="es_MX")  # opcional, nombre del mes en español

# Tabla mensual por estación
mensual = (
    base
    .groupby([station_col, "year", "month"], dropna=False)
    .agg(
        so2_max   = pd.NamedAgg(column="SO2",  aggfunc="max"),      # valor extremo (máximo)
        so2_median= pd.NamedAgg(column="SO2",  aggfunc="median"),   # mediana
        temp_mean = pd.NamedAgg(column="TOUT", aggfunc="mean"),     # promedio
        n_so2     = pd.NamedAgg(column="SO2",  aggfunc="count"),    # conteo no nulos
        n_temp    = pd.NamedAgg(column="TOUT", aggfunc="count")
    )
    .reset_index()
)

# (Opcional) añade el nombre del mes para lectura humana
mensual = mensual.merge(
    base[[station_col, "year", "month", "month_name"]].drop_duplicates(),
    on=[station_col, "year", "month"],
    how="left"
)

# Orden y formato
mensual = mensual.sort_values([station_col, "year", "month"]).reset_index(drop=True)
mensual[["so2_max","so2_median","temp_mean"]] = mensual[["so2_max","so2_median","temp_mean"]].round(2)

mensual''''

,Zona,year,month,so2_max,so2_median,temp_mean,n_so2,n_temp,month_name
0,CENTRO,2021,1,17.8,4.55,13.54,338,341,Enero
1,CENTRO,2021,2,14.2,4.10,14.84,229,285,Febrero
2,CENTRO,2021,3,11.3,8.90,19.66,143,340,Marzo
3,CENTRO,2021,4,11.4,1.60,21.97,250,328,Abril
4,CENTRO,2021,5,7.2,1.60,24.20,261,340,Mayo
...,...,...,...,...,...,...,...,...,...
751,SUROESTE2,2025,2,62.5,5.40,15.85,303,303,Febrero
752,SUROESTE2,2025,3,21.1,4.40,21.00,337,337,Marzo
753,SUROESTE2,2025,4,11.1,4.10,23.03,329,329,Abril
754,SUROESTE2,2025,5,12.4,3.60,25.53,341,341,Mayo
